In [1]:
import csv


ids = []
g = []
p = []

with open('train.csv', 'r') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        ids.append(row[0])
        
        gs = []
        for c in row[1].lower():
            if c.isalpha() or c == "'":
                gs.append(c)
        
        g.append(gs)
        ph = row[2].split()
        p.append(ph)
        
graphemes = g
phonemes = p

In [188]:
with open('data', 'w') as out:
    for i in range(1, len(ids)):
        for j in range(len(g[i])):
            if j > 0:
                out.write(' ')
            out.write(g[i][j])
        out.write('\t')
        for j in range(len(p[i])):
            if j > 0:
                out.write(' ')
            out.write(p[i][j])
        out.write('\n')

In [2]:
align_data = []
words = []

p_cls = set([])
g_cls = set([])

p_cls_cnt = {}
p_total_cnt = 0

def word_from_align(a):
    w = ""
    for chunk in a:
        if len(chunk) > 1:
            w += chunk[0] + chunk[-1]
        else:
            w += chunk
    return w

with open('data.m-mAlign.2-2.delX.1-best.conYX.align') as f:
    align_data = f.readlines()
    for i in range(len(align_data)):
        align_data[i] = align_data[i].split('\t')
        align_data[i][0] = align_data[i][0].split('|')
        align_data[i][1] = align_data[i][1].split('|')
        align_data[i][0].remove('')
        align_data[i][1].remove('\n')
        words.append(word_from_align(align_data[i][0]))
        
        for c in align_data[i][0]:
            g_cls.add(c)
        
        for c in align_data[i][1]:
            p_cls.add(c)

In [3]:
print (align_data[0], words[0])

p_cls_id = {}
g_cls_id = {}

p_cls_id['~'] = 0
i = 1
for pc in p_cls:
    p_cls_id[pc] = i
    i += 1
    
g_cls_id['~'] = 0
i = 1
for gc in g_cls:
    g_cls_id[gc] = i
    i += 1
    
print (len(p_cls_id), len(g_cls_id))

[['k:n', 'o', 'x', 'v', 'i', 'l:l', 'e'], ['N', 'AA', 'K:S', 'V', 'IH', 'L', '_']] knoxville
181 315


In [4]:
X = []
Y = []

def left_ctx(a, i, ctx_len=2):
    ctx = []
    j = i-1
    while j >= 0 and len(ctx) < ctx_len:
        if len(a[j]) > 1:
            if len(ctx) < ctx_len:
                ctx.append(a[j][-1])
            if len(ctx) < ctx_len:
                ctx.append(a[j][0])
        else:
            ctx.append(a[j])
        j -= 1
    
    while len(ctx) < ctx_len:
        ctx.append('~')
    
    return ctx[::-1]

def right_ctx(a, i, ctx_len=2):
    ctx = []
    j = i + 1
    while j < len(a) and len(ctx) < ctx_len:
        if len(a[j]) > 1:
            if len(ctx) < ctx_len:
                ctx.append(a[j][0])
            if len(ctx) < ctx_len:
                ctx.append(a[j][-1])
        else:
            ctx.append(a[j])
        j += 1
        
    while len(ctx) < ctx_len:
        ctx.append('~')
    
    return ctx

def get_xy(a, w):
    x = []
    y = []
    ri = 0
    for i in range(len(a)):
        if i == len(a) - 1 and len(a[i]) == 1:
            break
        l = []
        if len(a[i]) > 1:
            l = left_ctx(a, i) + [a[i][0], a[i][-1]] + right_ctx(a, i)
            
            ll = [c for c in l[1::]]
            if ri+4 < len(w):
                ll.append(w[ri+4])
            else:
                ll.append('~')
            ll = [ord(c) for c in ll]
            
            l = [ord(c) for c in l]
            x.append( l )
            y.append(1)
            x.append( ll )
            y.append(0)
        else:
            if len(a[i+1]) == 1:
                l = left_ctx(a, i) + [a[i], a[i+1]] + right_ctx(a, i+1)
            else:
                l = left_ctx(a, i) + [a[i], a[i+1][0], a[i+1][-1]] + right_ctx(a, i+1, 1)
            y.append(0)
            l = [ord(c) for c in l]
            x.append( l )
            
        ri += min(len(a[i]), 2)
        
    return (x, y)

for k in range(len(align_data)):
    xa, ya = get_xy(align_data[k][0], words[k])
    for x,y in zip(xa, ya):
        X.append(x)
        Y.append(y)

In [72]:
for i in range(20):
    print ([chr(c) for c in X[i]], Y[i])

['~', '~', 'k', 'n', 'o', 'x'] 1
['~', 'k', 'n', 'o', 'x', 'v'] 0
['k', 'n', 'o', 'x', 'v', 'i'] 0
['n', 'o', 'x', 'v', 'i', 'l'] 0
['o', 'x', 'v', 'i', 'l', 'l'] 0
['x', 'v', 'i', 'l', 'l', 'e'] 0
['v', 'i', 'l', 'l', 'e', '~'] 1
['i', 'l', 'l', 'e', '~', '~'] 0
['~', '~', 'm', 'o', 'v', 'i'] 0
['~', 'm', 'o', 'v', 'i', 'e'] 0
['m', 'o', 'v', 'i', 'e', 'g'] 0
['o', 'v', 'i', 'e', 'g', 'o'] 1
['v', 'i', 'e', 'g', 'o', 'i'] 0
['i', 'e', 'g', 'o', 'i', 'n'] 0
['e', 'g', 'o', 'i', 'n', 'g'] 0
['g', 'o', 'i', 'n', 'g', '~'] 0
['o', 'i', 'n', 'g', '~', '~'] 1
['i', 'n', 'g', '~', '~', '~'] 0
['~', '~', 'p', 'h', 'o', 't'] 1
['~', 'p', 'h', 'o', 't', 'o'] 0


In [5]:
from sklearn.ensemble import RandomForestClassifier


clf = RandomForestClassifier(n_estimators=10)
clf = clf.fit(X, Y)

In [6]:
def word_to_window(w, focus, l_ctx_len=2, r_ctx_len=2):
    lctx = []
    rctx = []
    j = focus - 1
    while j >= 0 and len(lctx) < l_ctx_len:
        lctx.append(w[j])
        j -= 1
    while len(lctx) < l_ctx_len:
        lctx.append('~')
    j = focus + 2
    while j < len(w) and len(rctx) < r_ctx_len:
        rctx.append(w[j])
        j += 1
    while len(rctx) < r_ctx_len:
        rctx.append('~')
    l = lctx[::-1] + [w[focus], w[focus+1]] + rctx
    l = [ord(c) for c in l]
    return l

In [7]:
def letter_chunking(w):
    f = []
    for focus in range(len(w)-1):
        window = word_to_window(w, focus)
        f.append(clf.predict([window]))
    f.append(0)
    for i in range(1, len(w)):
        if f[i] == 1 and f[i-1] == 1:
            f[i] = 0
    chunks = []
    i = 0
    while i < len(w):
        if f[i] == 1:
            chunks.append([w[i], w[i+1]])
            i += 2
        else:
            chunks.append([w[i]])
            i += 1
    return chunks

In [8]:
print (letter_chunking("THREEMONTH".lower()))

[['t', 'h'], ['r'], ['e', 'e'], ['m'], ['o'], ['n'], ['t', 'h']]


In [199]:
XX = []
YY = []

def get_xy2(g, p):
    x = []
    y = []
    for i in range(len(g)):
        l_ctx = []
        r_ctx = []
        j = i - 1
        while j >= 0 and len(l_ctx) < 4:
            l_ctx.append(g_cls_id[g[j]])
            l_ctx.append(p_cls_id[p[j]])
        while len(l_ctx) < 4:
            l_ctx.append(g_cls_id['~'])
            l_ctx.append(p_cls_id['_'])
            
        if i < len(g) - 1:
            r_ctx.append(g_cls_id[g[i+1]])
        else:
            r_ctx.append(g_cls_id['~'])
            
        window = l_ctx[::-1] + [g_cls_id[g[i]]] + r_ctx
        x.append(window)
        y.append(p_cls_id[p[i]])
        
    return (x, y)

for k in range(len(align_data)):
    xs, ys = get_xy2(align_data[k][0], align_data[k][1])
    for x, y in zip(xs, ys):
        XX.append(x)
        YY.append(y)

In [200]:
clf2 = RandomForestClassifier(n_estimators=10)
clf2 = clf2.fit(XX, YY)

In [8]:
g_from_cls = {}
p_from_cls = {}

for gr in g_cls_id:
    g_from_cls[g_cls_id[gr]] = gr
    
for ph in p_cls_id:
    p_from_cls[p_cls_id[ph]] = ph
    
print (g_cls_id['k'], g_from_cls[g_cls_id['k']])
print (phonemes[1], graphemes[1])

205 k
['N', 'AA', 'K', 'S', 'V', 'IH', 'L'] ['k', 'n', 'o', 'x', 'v', 'i', 'l', 'l', 'e']


In [202]:
def g2p(g):
    ch = letter_chunking(g.lower())
    g = []
    for c in ch:
        if len(c) > 1:
            g.append(c[0] + ':' + c[1])
        else:
            g.append(c[0])
    
    p = []
    for i in range(len(g)):
        l_ctx = []
        r_ctx = []
        j = i - 1
        while j >= 0 and len(l_ctx) < 4:
            l_ctx.append(g_cls_id.get(g[j], 0))
            l_ctx.append(p_cls_id[p[j]])
        while len(l_ctx) < 4:
            l_ctx.append(g_cls_id['~'])
            l_ctx.append(p_cls_id['_'])
            
        if i < len(g) - 1:
            r_ctx.append(g_cls_id.get(g[i+1], 0))
        else:
            r_ctx.append(g_cls_id['~'])
            
        window = l_ctx[::-1] + [g_cls_id.get(g[i], 0)] + r_ctx
        pp = p_from_cls[clf2.predict([window])[0]]
        p.append(pp)
        
    return p  

In [203]:
print (g2p('knoxville'))

['N', 'AA', 'K:S', 'V', 'IH', 'L', '_']


In [9]:
p_to_ch = {}

with open('phones.txt') as p_dct:
    lines = p_dct.readlines()
    for l in lines:
        ph, ch = l.split()
        p_to_ch[ph] = ch

In [10]:
def transform(ps):
    s = ""
    for ph in ps:
        if ':' in ph:
            a,b = ph.split(':')
            s += p_to_ch[a] + p_to_ch[b]
        elif ph == '_':
            continue
        else:
            s += p_to_ch[ph]
            
    return s

In [206]:
print(transform(g2p('knoxville')))

watCIqu


In [11]:
def test_g2p_model(g2p_model):
    test_words = []
    test_ids = []

    with open('test.csv', 'r') as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            test_ids.append(row[0])
            tw = ""
            for c in row[1].lower():
                if c.isalpha() or c == "'":
                    tw += c
            test_words.append(tw)
            
    with open('out.csv', 'w+') as out:
        out.write('Id,Transcription\n')
        for i in range(1, len(test_ids)):
            try:
                out.write(test_ids[i] + ',' + transform(g2p_model(test_words[i])) + '\n')
            except KeyError:
                print (test_words[i])

In [12]:
from nltk.probability import FreqDist
from nltk import bigrams
from nltk.probability import SimpleGoodTuringProbDist


p_fd = FreqDist()
for k in range(len(align_data)):
    p_fd.update(bigrams(align_data[k][1]))
    
p_lm = SimpleGoodTuringProbDist(p_fd)

In [212]:
XX3 = []
YY3 = []

def get_xy3(g, p):
    x = []
    y = []
    for i in range(len(g)):
        l_ctx = []
        r_ctx = []
        j = i - 1
        while j >= 0 and len(l_ctx) < 2:
            l_ctx.append(g_cls_id[g[j]])
        while len(l_ctx) < 2:
            l_ctx.append(g_cls_id['~'])
            
        if i < len(g) - 1:
            r_ctx.append(g_cls_id[g[i+1]])
        else:
            r_ctx.append(g_cls_id['~'])
            
        window = l_ctx[::-1] + [g_cls_id[g[i]]] + r_ctx
        x.append(window)
        y.append(p_cls_id[p[i]])
        
    return (x, y)

for k in range(len(align_data)):
    xs, ys = get_xy3(align_data[k][0], align_data[k][1])
    for x, y in zip(xs, ys):
        XX3.append(x)
        YY3.append(y)

In [213]:
clf3 = RandomForestClassifier(n_estimators=10)
clf3 = clf3.fit(XX3, YY3)

In [13]:
def Viterbi(cs, simple_transition=False):
    l = []
    
    for i in range(len(cs)):
        l.append([0] * len(cs[i]))
        
    for i in range(len(l[0])):
        l[0][i] = (-1, cs[0][i][1])
        
    for k in range(1,len(cs)):
        for i in range(len(cs[k])):
            emission = float(cs[k][i][1])
            mx = 239
            for j in range(len(cs[k-1])):
                prev = float(l[k-1][j][1])
                
                transition = 0.0
                if not simple_transition:
                    transition = float((p_lm.prob( (cs[k-1][j][0], cs[k][i][0]) )))
                else:
                    if p_fd[(cs[k-1][j][0], cs[k][i][0])] > 0:
                        transition = 1.0
                
                tmp = prev * transition * emission
                if tmp > mx or mx == 239:
                    mx = tmp
                    l[k][i] = (j, tmp)
                    
    mx = l[-1][0][1]
    mx_i = 0
    for i in range(len(l[-1])):
        if l[-1][i][1] > mx:
            mx = l[-1][i][1]
            mx_i = i
    
    seq = []    
    k = len(l) - 1
    while k >= 0:
        seq.append(cs[k][mx_i][0])
        mx_i = l[k][mx_i][0]
        k -= 1
        
    #for i in range(len(l)):
    #    print(l[i])
        
    return seq[::-1]

In [ ]:
def g2p_2(g):
    ch = letter_chunking(g.lower())
    g = []
    for c in ch:
        if len(c) > 1:
            g.append(c[0] + ':' + c[1])
        else:
            g.append(c[0])
    
    candidates = []
    for i in range(len(g)):
        l_ctx = []
        r_ctx = []
        j = i - 1
        while j >= 0 and len(l_ctx) < 2:
            l_ctx.append(g_cls_id.get(g[j], 0))
        while len(l_ctx) < 2:
            l_ctx.append(g_cls_id['~'])
            
        if i < len(g) - 1:
            r_ctx.append(g_cls_id.get(g[i+1], 0))
        else:
            r_ctx.append(g_cls_id['~'])
            
        window = l_ctx[::-1] + [g_cls_id.get(g[i], 0)] + r_ctx
        
        probs = clf3.predict_proba([window])[0]
        probs_s = []
        for i in range(len(probs)):
            if probs[i] > 0:
                probs_s.append((p_from_cls[i], (probs[i])))
                               
        candidates.append(probs_s)
        
    return Viterbi(np.array(candidates))  

In [254]:
print(g2p_2('home'), g2p('home'))
print(g2p_2('easy'), g2p('easy'))

['HH', 'AA', 'M', '_'] ['HH', 'OW', 'M', '_']
['IY', 'Z', 'IY'] ['IY', 'Z', 'IY']


In [39]:
def distance(a, b):
    n, m = len(a), len(b)
    if n > m:
        a, b = b, a
        n, m = m, n

    current_row = range(n+1) 
    for i in range(1, m+1):
        previous_row, current_row = current_row, [i]+[0]*n
        for j in range(1,n+1):
            add, delete, change = previous_row[j]+1, current_row[j-1]+1, previous_row[j-1]
            if a[j-1] != b[i-1]:
                change += 1
            current_row[j] = min(add, delete, change)

    return current_row[n]

def test(g2p_model,l=1000):
    sum_dist = 0
    for i in range(1, l): #
        y = transform(phonemes[i])
        gs = ""
        for c in graphemes[i]:
            gs += c
        y2 = transform(g2p_model(gs))
        sum_dist += distance(y, y2)
    
    return float(sum_dist)/float(l)

In [251]:
print(test(g2p,5000))

0.7968


In [252]:
print(test(g2p_2,5000))

0.9964


In [15]:
a_max_len = 0
for i in range(len(align_data)):
    a_max_len = max(a_max_len, len(align_data[i][0]))
    a_max_len = max(a_max_len, len(align_data[i][1]))
    
print (a_max_len)

31


In [16]:
import numpy as np

def one_hot(seq, a_cls, cls_cnt, ignore=True):
    vec = []
    for s_i in seq:
        v = []
        for c in s_i:
            vg = [0] * cls_cnt
            if c == '~':
                if not ignore:
                    vg[a_cls[c]] = 1
            else:
                cls = a_cls.get(c, -1)
                
                if cls == -1:
                    c = c[0]
                    
                vg[a_cls[c]] = 1
            v.append(np.array(vg))
        vec.append(np.array(v))
    vec = np.array(vec)
    return vec
    

def gen_data(max_len, begin,train_length, ignore=True):
    gs = []
    ps = []
    
    for i in range(begin, begin + train_length):
        gs_i = align_data[i][0]
        ps_i = align_data[i][1]
        if len(gs_i) < max_len:
            gs_i = gs_i + ['~'] * (max_len - len(gs_i))
        if len(ps_i) < max_len:
            ps_i = ps_i + ['~'] * (max_len - len(ps_i))
        gs.append(gs_i)
        ps.append(ps_i)
        
    return (one_hot(gs, g_cls_id, len(g_cls_id), ignore), one_hot(ps, p_cls_id, len(p_cls_id), ignore))   

In [17]:
X_train, Y_train = gen_data(a_max_len, 0, 20000, False) 

In [18]:
print (X_train.shape, Y_train.shape)

(20000, 31, 315) (20000, 31, 181)


In [19]:
from keras.models import Sequential
from keras.layers.core import Masking, Dense, Merge, TimeDistributedDense, Activation, RepeatVector
from keras.layers.recurrent import LSTM
from keras.layers.wrappers import TimeDistributed
from keras.optimizers import RMSprop

max_len = X_train.shape[1]
n_feats = X_train.shape[2]
n_outs = Y_train.shape[2] 
n_hidden = 128 # 32

Using TensorFlow backend.


In [20]:
model = Sequential([
    LSTM(512, return_sequences=True, input_shape=(max_len, n_feats)),
    LSTM(32, return_sequences=True),
    LSTM(32, return_sequences=True),
    TimeDistributed(Dense(n_outs, activation='softmax'))
])
optimizer = RMSprop(lr=0.001, decay=1e-6)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [37]:
model2 = Sequential([
    LSTM(n_hidden, return_sequences=True, input_length=max_len, input_shape=(max_len, n_feats)),
    LSTM(n_hidden, return_sequences=True),
    LSTM(n_hidden, return_sequences=False),
    Dense(n_hidden, activation='relu'),
 
    RepeatVector(max_len),
    LSTM(n_hidden, input_length=max_len, return_sequences=True),
    TimeDistributed(Dense(n_outs, activation='softmax'))
])
optimizer = RMSprop(lr=0.001, decay=1e-6)
model2.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [21]:
hist = model.fit(X_train, Y_train, batch_size=80, nb_epoch=25)

Epoch 1/25
20000/20000 [==============================] - 351s - loss: 1.2627 - categorical_accuracy: 0.7846   
Epoch 2/25
20000/20000 [==============================] - 353s - loss: 0.8194 - categorical_accuracy: 0.7980   
Epoch 3/25
20000/20000 [==============================] - 363s - loss: 0.7256 - categorical_accuracy: 0.8110   
Epoch 4/25
20000/20000 [==============================] - 365s - loss: 0.4687 - categorical_accuracy: 0.8902   
Epoch 5/25
20000/20000 [==============================] - 366s - loss: 0.2366 - categorical_accuracy: 0.9424   
Epoch 6/25
20000/20000 [==============================] - 355s - loss: 0.1589 - categorical_accuracy: 0.9552   
Epoch 7/25
20000/20000 [==============================] - 338s - loss: 0.1348 - categorical_accuracy: 0.9588   
Epoch 8/25
20000/20000 [==============================] - 325s - loss: 0.1235 - categorical_accuracy: 0.9604   
Epoch 9/25
20000/20000 [==============================] - 328s - loss: 0.1164 - categorical_accuracy: 0.

In [31]:
X_train2, Y_train2 = None, None
X_train2, Y_train2 = gen_data(a_max_len, 40000, 20000, False)

In [32]:
hist = model.fit(X_train2, Y_train2, batch_size=80, nb_epoch=25)

Epoch 1/25
20000/20000 [==============================] - 366s - loss: 0.1086 - categorical_accuracy: 0.9630   
Epoch 2/25
20000/20000 [==============================] - 352s - loss: 0.0934 - categorical_accuracy: 0.9672   
Epoch 3/25
20000/20000 [==============================] - 355s - loss: 0.0855 - categorical_accuracy: 0.9700   
Epoch 4/25
20000/20000 [==============================] - 374s - loss: 0.0797 - categorical_accuracy: 0.9722   
Epoch 5/25
20000/20000 [==============================] - 303s - loss: 0.0752 - categorical_accuracy: 0.9740   
Epoch 6/25
20000/20000 [==============================] - 307s - loss: 0.0716 - categorical_accuracy: 0.9752   
Epoch 7/25
20000/20000 [==============================] - 349s - loss: 0.0689 - categorical_accuracy: 0.9762   
Epoch 8/25
20000/20000 [==============================] - 472s - loss: 0.0666 - categorical_accuracy: 0.9769   
Epoch 9/25
20000/20000 [==============================] - 367s - loss: 0.0649 - categorical_accuracy: 0.

In [42]:
def word2vec(w, max_len, a_cls, alpha_len, ignore):
    w = [c for c in w]
    if len(w) < max_len:
        w = w + ['~'] * (max_len - len(w))
    return one_hot([w], a_cls, alpha_len, ignore)

def g2vec(w, max_len, ignore):
    return word2vec(w, max_len, g_cls_id, len(g_cls_id),ignore)

def p2vec(w, max_len, ignore):
    return word2vec(w, max_len, p_cls_id, len(p_cls_id),ignore)

def build_candidates(probs):
    pis = []
    for plst in probs:
        pi = []
        i = 0
        for p in plst:
            pi.append((p_from_cls[i], p))
            i += 1
        pi = sorted(pi, key=lambda x: -x[1])
        pis.append(pi[:1])
    return pis

In [24]:
def g2p_rnn(w, simple_transition=False, viterbi=True, g2p_model=model, bidirect=False):
    ch = letter_chunking(w.lower())
    g = []
    for c in ch:
        if len(c) > 1:
            g.append(c[0] + ':' + c[1])
        else:
            g.append(c[0])
            
    inp = g2vec(g, a_max_len, False)
    ans = []
    if not bidirect:
        ans = g2p_model.predict(inp)
    else:
        ans = g2p_model.predict([inp, inp])
    ps = []
    if viterbi:
        for ph in Viterbi(build_candidates(ans[0]), simple_transition):
            if ph == '~':
                break
            elif ph == '_':
                continue
            ps.append(ph)
    else:
        for lst in ans[0]:
            ch = p_from_cls[np.argmax(lst)]
            if ch == '~':
                break
            elif ch == '_':
                continue
            ps.append(ch)
            
    return ps

In [44]:
test_g2p_model(g2p_rnn)

In [43]:
print(g2p_rnn('knoxville'))
print (test(g2p_rnn, 100))

['N', 'AA', 'K:S', 'V', 'IH', 'L']
1.15


In [41]:
def fork (model, n=2):
    forks = []
    for i in range(n):
        f = Sequential()
        f.add (model)
        forks.append(f)
    return forks

left = Sequential()
left.add(LSTM(output_dim=n_hidden, init='uniform', inner_init='uniform',
              forget_bias_init='one', return_sequences=True, activation='tanh',
              inner_activation='sigmoid', input_shape=(max_len, n_feats)))
         
right = Sequential()
right.add(LSTM(output_dim=n_hidden, init='uniform', inner_init='uniform',
               forget_bias_init='one', return_sequences=True, activation='tanh',
               inner_activation='sigmoid', input_shape=(max_len, n_feats), go_backwards=True))

model2 = Sequential()
model2.add(Merge([left, right], mode='sum'))

#Add second Bidirectional LSTM layer

# left, right = fork(model2)

# left.add(LSTM(output_dim=n_hidden, init='uniform', inner_init='uniform',
#               forget_bias_init='one', return_sequences=True, activation='tanh',
#               inner_activation='sigmoid'))

# right.add(LSTM(output_dim=n_hidden, init='uniform', inner_init='uniform',
#                forget_bias_init='one', return_sequences=True, activation='tanh',
#                inner_activation='sigmoid',  go_backwards=True))


# model2 = Sequential()
# model2.add(Merge([left, right], mode='sum'))

model2.add(TimeDistributedDense(n_outs))
model2.add(Activation('softmax'))

optimizer = RMSprop(lr=0.001, decay=1e-6)
model2.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

/home/austud/anaconda3/lib/python3.5/site-packages/keras/layers/core.py:1112: UserWarning: TimeDistributedDense is deprecated, please use TimeDistributed(Dense(...)) instead.
  warnings.warn('TimeDistributedDense is deprecated, '


In [46]:
def g2p_drnn(w):
    return g2p_rnn(w, g2p_model=model2, bidirect=True)

In [47]:
model2.fit([X_train, X_train], Y_train, batch_size=80, nb_epoch=20)

Epoch 1/20
20000/20000 [==============================] - 578s - loss: 0.6808 - categorical_accuracy: 0.8663   
Epoch 2/20
20000/20000 [==============================] - 587s - loss: 0.1638 - categorical_accuracy: 0.9532   
Epoch 3/20
20000/20000 [==============================] - 589s - loss: 0.1254 - categorical_accuracy: 0.9579   
Epoch 4/20
20000/20000 [==============================] - 591s - loss: 0.1155 - categorical_accuracy: 0.9601   
Epoch 5/20
20000/20000 [==============================] - 591s - loss: 0.1092 - categorical_accuracy: 0.9617   
Epoch 6/20
20000/20000 [==============================] - 589s - loss: 0.1037 - categorical_accuracy: 0.9633   
Epoch 7/20
20000/20000 [==============================] - 593s - loss: 0.0986 - categorical_accuracy: 0.9649   
Epoch 8/20
20000/20000 [==============================] - 592s - loss: 0.0941 - categorical_accuracy: 0.9664   
Epoch 9/20
20000/20000 [==============================] - 592s - loss: 0.0896 - categorical_accuracy: 0.